<a href="https://colab.research.google.com/github/xy2119/Causal_Knowledge_GNN/blob/main/notebooks/3_causal_without_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook uses GCN and GAT to estimate average treatment effect based on BN Connectivity and Causally Weighted Feature Embeddings
Dataset created without saving to disk (avoid `InMemoryDataset` class)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from gensim.models import word2vec
from causalml.metrics import *
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

/Users/patricia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/patricia/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [ ]:
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
df_train=pd.read_csv("criteo_sampled/criteo_train.csv",index_col=0)
df_test=pd.read_csv("criteo_sampled/criteo_test.csv",index_col=0)

In [ ]:
# load causal weighting
ate_list=[]
ate=pd.read_excel("criteo_sampled/feats_ate_x13.xlsx",index_col=0)
for i in [c for c in df_train.columns[:-4]]+["visit","treatment"]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
ate_list

[-0.01388679662724193,
 0.05071538371534517,
 0.001328853128405821,
 -0.009331114629769072,
 0.1933108932875426,
 0.0008912893595676433,
 -0.0002958249515911775,
 -0.009201954095841196,
 0.1034574925533489,
 0.0741312293943781,
 -0.01664079180275199,
 0.1331562899243786,
 0.9310119697738223,
 0.007447354046319239]

In [ ]:
# load importance weighting
imp_list=[]
imp=pd.read_excel("criteo_sampled/feats_imp_criteo.xlsx",index_col=0)
for i in [c for c in df_train.columns[:-3]]+["visit" ]:
    imp_list.append(float(imp[imp['Feature']==i]["Importance"].values))
imp_list

[2.156023947463837e-05,
 0.0,
 0.0,
 0.0003188499540556222,
 0.0001497909397585317,
 0.0,
 8.849588630255312e-05,
 0.0,
 4.08560226787813e-05,
 0.0,
 2.923487772932276e-05,
 0.0,
 0.9985944032669067,
 0.0007568388828076422]

In [ ]:
# load edge index
edge_index=pd.read_csv('criteo_sampled/edge_index_criteo.csv')
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))
edge_index

tensor([[ 8,  8,  8,  9,  9,  9,  9,  6,  6,  6,  6,  6,  6,  6,  6,  2,  2,  2,
          2,  1,  1,  1, 12,  1,  0,  0,  0,  0,  0,  0,  0,  1, 11,  1,  1,  5,
          5,  5,  5,  5,  5,  7,  7,  7,  3,  3,  3,  9,  5,  3,  4,  7, 10,  1,
          8,  9, 12,  4,  5, 10,  3,  1,  4,  7, 10,  1,  8,  9,  4, 10,  1,  9,
          6,  2, 12,  4, 10,  1,  4, 10,  3,  1,  2, 11,  4, 10,  3,  1,  4, 10,
          1,  4, 10,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
         13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
        [ 9,  5,  3,  4,  7, 10,  1,  8,  9, 12,  4,  5, 10,  3,  1,  4,  7, 10,
          1,  8,  9,  4, 10,  1,  9,  6,  2, 12,  4, 10,  1,  4, 10,  3,  1,  2,
         11,  4, 10,  3,  1,  4, 10,  1,  4, 10,  1,  8,  8,  8,  9,  9,  9,  9,
          6,  6,  6,  6,  6,  6,  6,  6,  2,  2,  2,  2,  1,  1,  1, 12,  1,  0,
          0,  0,  0,  0,  0,  0,  1, 11,  1,  1,  5,  5,  5,  5,  5,  5,  7,  7,
          7,  3,  3,  3, 13, 13, 13, 13, 13, 13, 13

In [ ]:
# load node embedding
model_dw=word2vec.Word2Vec.load("criteo_sampled/deepwalk_10d_x13.model")
# model_n2v=word2vec.Word2Vec.load("criteo_sampled/Node2Vec_10d_x13.model")

lst_dw=[]
# lst_n2v=[]
for i in range(14):
    lst_dw.append(model_dw.wv[i])
    #lst_n2v.append(model_n2v.wv[i])
len(lst_dw)

14

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
# Original Table of Results
result=pd.DataFrame(columns=[['AUUC']])
result.loc['S Learner(LR)','AUUC']=0.497983
result.loc['S Learner(XGB)','AUUC']=0.875572
result.loc['S Learner(LGBM)','AUUC']=0.883033

result.loc['GCN (Struct)','AUUC']=0.501865
result.loc['GCN (Struct+Feature)','AUUC']=0.721959
result.loc['GCN (Struct+Causal Weighting)','AUUC']=0.732616


result.loc['GAT (Struct)','AUUC']=0.544286
result.loc['GAT (Struct+Feature)','AUUC']=0.847630
result.loc['GAT (Struct+Causal Weighting)','AUUC']=0.8807

result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807


In [ ]:
# Input the feature mode and get corresponding dimensions

feats_mode='causal+imp'


if feats_mode=='causal+imp':
    in_dim=39  # 1+10+14+14

elif feats_mode =='noweighting':
    in_dim=11  # 1+10

elif feats_mode in ['causal*imp','causal','imp','equal']:
    in_dim=25  # 1+10+14
    
else:
    print('Error: Input Dimension Error')
    
# subset data for demo, delete this while training ob full dataset
df_train = df_train[:1000]
df_test = df_test[:1000]

col=df_train.columns


data_list=[]
weighted_feats=[]

# Set no. of folds on dataset
folds= 10  # 10 folds
batch=int(len(df_train)/folds) # batch size 

for f in range(folds):
    train=df_train[(batch*f):(batch*(f+1))]
    x_train=torch.from_numpy(np.array(train[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
    y_train=torch.from_numpy(np.array(train['y'])).reshape(train.shape[0],1).to(torch.float32)

    for i in range(x_train.shape[0]):
        Edge_index = edge_index.type(torch.long)
        X =x_train[i]
        
        if feats_mode =='causal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted) # feature (10d), causally weighted features        
                                                    ))) 
        elif feats_mode =='imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            weighted_feats.append(imp_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(imp_weighted) # feature (10d),importance weighted features        
                                                    ))) 
        elif feats_mode =='causal*imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.multiply(np.multiply(np.array(X),np.array(imp_list)),np.array(ate_list))
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='causal+imp':
            t=torch.zeros(14,39) # 39 dimensions = 1+10+14+14 = node number, node embedding, causal features, important features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(imp_weighted+causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted),# feature (10d), causality weighted features
                                                    list(imp_weighted)# feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='equal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.array(X)
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), equally weighted features        
                                                    ))) 
        else:
            t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


            for j in range(14):
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    ))) 

        Y = y_train[i].reshape(-1,1).to(torch.float32)
        data = Data(x=t, edge_index=Edge_index, y=Y)
        data_list.append(data)
train_loader =DataLoader(data_list, batch_size=200, shuffle=False,num_workers=0)


y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)

data_list=[]
weighted_feats=[]

for f in range(folds):

    test=df_test[(batch*f):(batch*(f+1))]
    x_test=torch.from_numpy(np.array(test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
    y_test=torch.from_numpy(np.array(test['y'])).reshape(test.shape[0],1).to(torch.float32)

    for i in range(x_test.shape[0]):
        Edge_index = edge_index.type(torch.long)
        X =x_test[i]

        if feats_mode =='causal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted) # feature (10d), causally weighted features        
                                                    ))) 
        elif feats_mode =='imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            weighted_feats.append(imp_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(imp_weighted) # feature (10d),importance weighted features        
                                                    ))) 
        elif feats_mode =='causal*imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.multiply(np.multiply(np.array(X),np.array(imp_list)),np.array(ate_list))
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='causal+imp':
            t=torch.zeros(14,39) # 39 dimensions = 1+10+14+14 = node number, node embedding, causal features, important features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(imp_weighted+causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted),# feature (10d), causality weighted features
                                                    list(imp_weighted)# feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='equal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.array(X)
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), equally weighted features        
                                                    ))) 
        else:
            t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


            for j in range(14):
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    ))) 

        Y = y_test[i].reshape(-1,1).to(torch.float32)
        data = Data(x=t, edge_index=Edge_index, y=Y)
        data_list.append(data)
test_loader =DataLoader(data_list, batch_size=200, shuffle=False,num_workers=0)

# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_dim, 64)
        self.conv2 = GCNConv(64, 10)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(train_loader):
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(df_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(test_loader):
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(df_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

In [ ]:
num_epochs = 2560
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in iter(test_loader):
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-2.908061  MSE:0.015570  KL:3.440364  MAE:0.100714  RMSE:0.124779  CVRMSE:31.194666  train_loss:
0.01556971613317728
R2:-0.494224  MSE:0.004469  KL:5.698337  MAE:0.044478  RMSE:0.066852  CVRMSE:22.284086  test_loss:
2.2346122306771575e-05
R2:-0.374339  MSE:0.005475  KL:3.332566  MAE:0.035122  RMSE:0.073996  CVRMSE:18.498932  R2:-0.292356  MSE:0.005149  KL:3.348468  MAE:0.030706  RMSE:0.071755  CVRMSE:17.938691  R2:-0.122749  MSE:0.004473  KL:3.603329  MAE:0.026905  RMSE:0.066881  CVRMSE:16.720185  R2:-0.120190  MSE:0.004463  KL:4.105466  MAE:0.025523  RMSE:0.066804  CVRMSE:16.701117  R2:-0.062155  MSE:0.004232  KL:3.634430  MAE:0.019868  RMSE:0.065051  CVRMSE:16.262735  train_loss:
0.0042316249164287
R2:0.007847  MSE:0.002968  KL:0.603239  MAE:0.011034  RMSE:0.054475  CVRMSE:18.158345  test_loss:
1.4837646922387649e-05
R2:-0.026566  MSE:0.004090  KL:3.250882  MAE:0.015840  RMSE:0.063952  CVRMSE:15.987959  R2:-0.034696  MSE:0.004122  KL:2.936221  MAE:0.013471  RMSE:0.064205  CVRMSE:1

R2:0.117864  MSE:0.003514  KL:0.122063  MAE:0.008886  RMSE:0.059283  CVRMSE:14.820656  R2:0.069812  MSE:0.003706  KL:0.085802  MAE:0.008235  RMSE:0.060876  CVRMSE:15.218962  R2:0.084583  MSE:0.003647  KL:0.068984  MAE:0.008189  RMSE:0.060391  CVRMSE:15.097646  R2:0.118763  MSE:0.003511  KL:0.063959  MAE:0.007876  RMSE:0.059252  CVRMSE:14.813101  R2:0.165962  MSE:0.003323  KL:0.061132  MAE:0.007667  RMSE:0.057644  CVRMSE:14.410947  train_loss:
0.003322806270443834
R2:0.115118  MSE:0.002647  KL:0.034933  MAE:0.006709  RMSE:0.051446  CVRMSE:17.148634  test_loss:
1.3233405610662885e-05
R2:0.094332  MSE:0.003608  KL:0.088619  MAE:0.008576  RMSE:0.060068  CVRMSE:15.017035  R2:0.060457  MSE:0.003743  KL:0.082488  MAE:0.008996  RMSE:0.061181  CVRMSE:15.295299  R2:0.102251  MSE:0.003577  KL:0.068393  MAE:0.008354  RMSE:0.059805  CVRMSE:14.951236  R2:0.091780  MSE:0.003618  KL:0.056414  MAE:0.007735  RMSE:0.060153  CVRMSE:15.038178  R2:0.105252  MSE:0.003565  KL:0.094235  MAE:0.008858  RMSE:0.05

R2:0.066590  MSE:0.003719  KL:0.058717  MAE:0.008439  RMSE:0.060981  CVRMSE:15.245299  R2:0.082922  MSE:0.003654  KL:0.060904  MAE:0.008092  RMSE:0.060445  CVRMSE:15.111333  R2:0.100674  MSE:0.003583  KL:0.044700  MAE:0.007907  RMSE:0.059857  CVRMSE:14.964366  R2:0.088255  MSE:0.003632  KL:0.061769  MAE:0.007852  RMSE:0.060269  CVRMSE:15.067335  R2:0.102949  MSE:0.003574  KL:0.068961  MAE:0.008220  RMSE:0.059782  CVRMSE:14.945422  train_loss:
0.003573849791428074
R2:0.113245  MSE:0.002652  KL:0.038173  MAE:0.006402  RMSE:0.051500  CVRMSE:17.166781  test_loss:
1.3261425352538936e-05
R2:0.124950  MSE:0.003486  KL:0.065308  MAE:0.008355  RMSE:0.059044  CVRMSE:14.761013  R2:0.078797  MSE:0.003670  KL:0.058840  MAE:0.008186  RMSE:0.060581  CVRMSE:15.145283  R2:0.124342  MSE:0.003489  KL:0.064926  MAE:0.007956  RMSE:0.059065  CVRMSE:14.766142  R2:0.104044  MSE:0.003569  KL:0.060977  MAE:0.008289  RMSE:0.059745  CVRMSE:14.936301  R2:0.103424  MSE:0.003572  KL:0.047500  MAE:0.007715  RMSE:0.05

R2:0.064706  MSE:0.003726  KL:0.041890  MAE:0.008309  RMSE:0.061043  CVRMSE:15.260671  R2:0.107516  MSE:0.003556  KL:0.059740  MAE:0.007718  RMSE:0.059629  CVRMSE:14.907327  R2:0.117406  MSE:0.003516  KL:0.045557  MAE:0.007739  RMSE:0.059298  CVRMSE:14.824507  R2:0.095793  MSE:0.003602  KL:0.041743  MAE:0.007611  RMSE:0.060020  CVRMSE:15.004918  R2:0.084522  MSE:0.003647  KL:0.047863  MAE:0.007862  RMSE:0.060393  CVRMSE:15.098146  train_loss:
0.003647264403116424
R2:0.106845  MSE:0.002671  KL:0.041724  MAE:0.006103  RMSE:0.051686  CVRMSE:17.228619  test_loss:
1.3357140647713095e-05
R2:0.100948  MSE:0.003582  KL:0.067639  MAE:0.007492  RMSE:0.059848  CVRMSE:14.962082  R2:0.105369  MSE:0.003564  KL:0.065588  MAE:0.007955  RMSE:0.059701  CVRMSE:14.925249  R2:0.106319  MSE:0.003560  KL:0.067845  MAE:0.008027  RMSE:0.059669  CVRMSE:14.917326  R2:0.103050  MSE:0.003573  KL:0.085731  MAE:0.008133  RMSE:0.059778  CVRMSE:14.944581  R2:0.078020  MSE:0.003673  KL:0.082387  MAE:0.007982  RMSE:0.06

R2:0.111912  MSE:0.003538  KL:0.059287  MAE:0.007727  RMSE:0.059482  CVRMSE:14.870569  R2:0.129947  MSE:0.003466  KL:0.051091  MAE:0.007334  RMSE:0.058875  CVRMSE:14.718800  R2:0.128180  MSE:0.003473  KL:0.056740  MAE:0.008111  RMSE:0.058935  CVRMSE:14.733739  R2:0.109676  MSE:0.003547  KL:0.076655  MAE:0.008117  RMSE:0.059557  CVRMSE:14.889276  R2:0.106724  MSE:0.003559  KL:0.059415  MAE:0.008565  RMSE:0.059656  CVRMSE:14.913943  train_loss:
0.003558811113180127
R2:0.111988  MSE:0.002656  KL:0.040777  MAE:0.006494  RMSE:0.051537  CVRMSE:17.178937  test_loss:
1.3280213897814974e-05
R2:0.115774  MSE:0.003523  KL:0.060326  MAE:0.008127  RMSE:0.059353  CVRMSE:14.838206  R2:0.129016  MSE:0.003470  KL:0.058406  MAE:0.007467  RMSE:0.058907  CVRMSE:14.726678  R2:0.123156  MSE:0.003493  KL:0.064170  MAE:0.008344  RMSE:0.059105  CVRMSE:14.776137  R2:0.123680  MSE:0.003491  KL:0.070634  MAE:0.008322  RMSE:0.059087  CVRMSE:14.771722  R2:0.128452  MSE:0.003472  KL:0.059232  MAE:0.008458  RMSE:0.05

...

R2:0.171316  MSE:0.003301  KL:0.044312  MAE:0.007707  RMSE:0.057458  CVRMSE:14.364624  R2:0.178112  MSE:0.003274  KL:0.043002  MAE:0.007727  RMSE:0.057222  CVRMSE:14.305598  R2:0.166158  MSE:0.003322  KL:0.037402  MAE:0.007660  RMSE:0.057637  CVRMSE:14.409256  R2:0.184988  MSE:0.003247  KL:0.044055  MAE:0.007522  RMSE:0.056983  CVRMSE:14.245633  R2:0.171780  MSE:0.003300  KL:0.039879  MAE:0.007433  RMSE:0.057442  CVRMSE:14.360594  train_loss:
0.003299626688385615
R2:0.136870  MSE:0.002582  KL:0.043826  MAE:0.006191  RMSE:0.050810  CVRMSE:16.936559  test_loss:
1.2908115662867204e-05
R2:0.179990  MSE:0.003267  KL:0.030093  MAE:0.008085  RMSE:0.057157  CVRMSE:14.289242  R2:0.197523  MSE:0.003197  KL:0.042688  MAE:0.007946  RMSE:0.056543  CVRMSE:14.135655  R2:0.102107  MSE:0.003577  KL:0.042148  MAE:0.008289  RMSE:0.059810  CVRMSE:14.952439  R2:0.121618  MSE:0.003499  KL:0.026803  MAE:0.007633  RMSE:0.059156  CVRMSE:14.789084  R2:0.171434  MSE:0.003301  KL:0.033571  MAE:0.007125  RMSE:0.05

R2:0.123148  MSE:0.003493  KL:0.033669  MAE:0.008776  RMSE:0.059105  CVRMSE:14.776200  R2:0.159783  MSE:0.003347  KL:0.036737  MAE:0.008102  RMSE:0.057857  CVRMSE:14.464235  R2:0.162269  MSE:0.003338  KL:0.024605  MAE:0.007042  RMSE:0.057771  CVRMSE:14.442817  R2:0.159920  MSE:0.003347  KL:0.030078  MAE:0.007445  RMSE:0.057852  CVRMSE:14.463054  R2:0.135095  MSE:0.003446  KL:0.044134  MAE:0.007685  RMSE:0.058701  CVRMSE:14.675195  train_loss:
0.0034457819696399384
R2:0.132516  MSE:0.002595  KL:0.040371  MAE:0.005828  RMSE:0.050938  CVRMSE:16.979215  test_loss:
1.2973218035767786e-05
R2:0.159410  MSE:0.003349  KL:0.041924  MAE:0.007681  RMSE:0.057870  CVRMSE:14.467442  R2:0.150779  MSE:0.003383  KL:0.035140  MAE:0.007977  RMSE:0.058166  CVRMSE:14.541529  R2:0.143912  MSE:0.003411  KL:0.035135  MAE:0.007432  RMSE:0.058401  CVRMSE:14.600200  R2:0.160759  MSE:0.003344  KL:0.036747  MAE:0.007431  RMSE:0.057823  CVRMSE:14.455829  R2:0.167259  MSE:0.003318  KL:0.029912  MAE:0.007426  RMSE:0.0

R2:0.198980  MSE:0.003191  KL:0.035662  MAE:0.007607  RMSE:0.056491  CVRMSE:14.122820  R2:0.177233  MSE:0.003278  KL:0.036862  MAE:0.007782  RMSE:0.057253  CVRMSE:14.313244  R2:0.099864  MSE:0.003586  KL:0.026117  MAE:0.007905  RMSE:0.059884  CVRMSE:14.971100  R2:0.161458  MSE:0.003341  KL:0.034653  MAE:0.007295  RMSE:0.057799  CVRMSE:14.449812  R2:0.184859  MSE:0.003248  KL:0.035673  MAE:0.007403  RMSE:0.056987  CVRMSE:14.246755  train_loss:
0.0032475206855451686
R2:0.140427  MSE:0.002571  KL:0.039351  MAE:0.006387  RMSE:0.050705  CVRMSE:16.901625  test_loss:
1.2854923144914209e-05
R2:0.165121  MSE:0.003326  KL:0.039801  MAE:0.007840  RMSE:0.057673  CVRMSE:14.418213  R2:0.186939  MSE:0.003239  KL:0.028024  MAE:0.008082  RMSE:0.056914  CVRMSE:14.228569  R2:0.136065  MSE:0.003442  KL:0.027854  MAE:0.007760  RMSE:0.058668  CVRMSE:14.666960  R2:0.150327  MSE:0.003385  KL:0.035412  MAE:0.007525  RMSE:0.058182  CVRMSE:14.545399  R2:0.167053  MSE:0.003318  KL:0.031776  MAE:0.007621  RMSE:0.0

R2:0.168167  MSE:0.003314  KL:0.036545  MAE:0.007709  RMSE:0.057568  CVRMSE:14.391887  R2:0.160380  MSE:0.003345  KL:0.042083  MAE:0.007700  RMSE:0.057836  CVRMSE:14.459093  R2:0.176550  MSE:0.003281  KL:0.038445  MAE:0.007503  RMSE:0.057277  CVRMSE:14.319180  R2:0.182287  MSE:0.003258  KL:0.041028  MAE:0.007698  RMSE:0.057077  CVRMSE:14.269218  R2:0.145694  MSE:0.003404  KL:0.046482  MAE:0.008150  RMSE:0.058340  CVRMSE:14.584995  train_loss:
0.0034035534867143725
R2:0.134119  MSE:0.002590  KL:0.039224  MAE:0.006269  RMSE:0.050891  CVRMSE:16.963525  test_loss:
1.294925413094461e-05
R2:0.141634  MSE:0.003420  KL:0.041563  MAE:0.007975  RMSE:0.058478  CVRMSE:14.619617  R2:0.171227  MSE:0.003302  KL:0.029245  MAE:0.007131  RMSE:0.057462  CVRMSE:14.365394  R2:0.172758  MSE:0.003296  KL:0.038352  MAE:0.007511  RMSE:0.057408  CVRMSE:14.352121  R2:0.128168  MSE:0.003473  KL:0.037534  MAE:0.008102  RMSE:0.058935  CVRMSE:14.733848  R2:0.191470  MSE:0.003221  KL:0.031963  MAE:0.007748  RMSE:0.05

R2:0.135252  MSE:0.003445  KL:0.021190  MAE:0.007741  RMSE:0.058695  CVRMSE:14.673866  R2:0.167141  MSE:0.003318  KL:0.043435  MAE:0.007940  RMSE:0.057603  CVRMSE:14.400761  R2:0.174676  MSE:0.003288  KL:0.026814  MAE:0.007738  RMSE:0.057342  CVRMSE:14.335465  R2:0.207086  MSE:0.003159  KL:0.029621  MAE:0.007006  RMSE:0.056205  CVRMSE:14.051177  R2:0.167096  MSE:0.003318  KL:0.025504  MAE:0.007749  RMSE:0.057605  CVRMSE:14.401148  train_loss:
0.0033182886822032743
R2:0.138513  MSE:0.002577  KL:0.032726  MAE:0.006199  RMSE:0.050761  CVRMSE:16.920428  test_loss:
1.2883538147434592e-05
R2:0.211321  MSE:0.003142  KL:0.033357  MAE:0.007580  RMSE:0.056054  CVRMSE:14.013603  R2:0.205729  MSE:0.003164  KL:0.029133  MAE:0.007770  RMSE:0.056253  CVRMSE:14.063196  R2:0.195714  MSE:0.003204  KL:0.027363  MAE:0.007844  RMSE:0.056606  CVRMSE:14.151582  R2:0.120288  MSE:0.003505  KL:0.034126  MAE:0.008070  RMSE:0.059201  CVRMSE:14.800281  R2:0.176738  MSE:0.003280  KL:0.028369  MAE:0.007088  RMSE:0.0

In [ ]:
# estimate area under the uplift curve (AUUC)
uplift=df_test.copy()
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T')
gcn_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gcn_auuc

,auuc
y_hat,0.777582
Random,0.489027


In [ ]:
print('Feature mode on GCN:', feats_mode)
print('MSE:',MSE(uplift['y'],uplift['y_hat']))

Feature mode on GCN: causal+imp
MSE: 0.00259513338732548


In [ ]:

if feats_mode == 'imp':
    result.loc['GCN (Struct+Important Weighting','AUUC']=auuc["y_hat"]
if feats_mode == 'causal+imp':
    result.loc['GCN (Struct+Causal+Important Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'causal*imp':
    result.loc['GCN (Struct+(Causal*Important) Weighting','AUUC']=auuc["y_hat"]
result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807
GCN (Struct+Causal+Important Weighting),0.777582


# GAT 
GCN + Attention Layer

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=in_dim,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(train_loader):
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(df_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(test_loader):
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(df_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl



num_epochs = 2560
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in iter(test_loader):
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-25.335778  MSE:0.104922  KL:2.934289  MAE:0.231009  RMSE:0.323916  CVRMSE:80.979063  train_loss:
0.10492173954844475
R2:-5.547298  MSE:0.019583  KL:5.300995  MAE:0.128627  RMSE:0.139939  CVRMSE:46.646385  test_loss:
9.791483543813229e-05
R2:-4.704450  MSE:0.022727  KL:2.975788  MAE:0.112224  RMSE:0.150753  CVRMSE:37.688300  R2:-2.593554  MSE:0.014317  KL:3.096289  MAE:0.077948  RMSE:0.119652  CVRMSE:29.913121  R2:-2.125688  MSE:0.012453  KL:3.174985  MAE:0.072824  RMSE:0.111592  CVRMSE:27.897962  R2:-1.425888  MSE:0.009665  KL:3.316360  MAE:0.064801  RMSE:0.098309  CVRMSE:24.577348  R2:-1.203343  MSE:0.008778  KL:3.245919  MAE:0.051748  RMSE:0.093692  CVRMSE:23.422904  train_loss:
0.00877811862155795
R2:-0.280573  MSE:0.003830  KL:4.146775  MAE:0.029076  RMSE:0.061889  CVRMSE:20.629518  test_loss:
1.915096596349031e-05
R2:-0.535084  MSE:0.006116  KL:3.096771  MAE:0.042126  RMSE:0.078203  CVRMSE:19.550851  R2:-0.519482  MSE:0.006054  KL:3.467679  MAE:0.042579  RMSE:0.077805  CVRMSE:

R2:0.073197  MSE:0.003692  KL:1.906742  MAE:0.012750  RMSE:0.060765  CVRMSE:15.191247  R2:0.055547  MSE:0.003763  KL:2.069550  MAE:0.013575  RMSE:0.061341  CVRMSE:15.335211  R2:0.064832  MSE:0.003726  KL:2.190748  MAE:0.013246  RMSE:0.061039  CVRMSE:15.259647  R2:0.079990  MSE:0.003665  KL:1.953211  MAE:0.013357  RMSE:0.060542  CVRMSE:15.135473  R2:0.030448  MSE:0.003863  KL:2.637734  MAE:0.013480  RMSE:0.062151  CVRMSE:15.537648  train_loss:
0.003862696239957586
R2:0.060736  MSE:0.002809  KL:1.772939  MAE:0.008871  RMSE:0.053003  CVRMSE:17.667730  test_loss:
1.4046691547264344e-05
R2:0.078295  MSE:0.003672  KL:1.731978  MAE:0.012444  RMSE:0.060598  CVRMSE:15.149407  R2:0.053030  MSE:0.003773  KL:2.020650  MAE:0.012861  RMSE:0.061423  CVRMSE:15.355638  R2:0.073174  MSE:0.003692  KL:1.819657  MAE:0.011708  RMSE:0.060766  CVRMSE:15.191431  R2:0.074954  MSE:0.003685  KL:2.143875  MAE:0.011791  RMSE:0.060707  CVRMSE:15.176835  R2:0.074131  MSE:0.003689  KL:1.791869  MAE:0.012531  RMSE:0.06

R2:0.063399  MSE:0.003731  KL:1.745479  MAE:0.011737  RMSE:0.061085  CVRMSE:15.271333  R2:0.051365  MSE:0.003779  KL:2.214662  MAE:0.012121  RMSE:0.061477  CVRMSE:15.369132  R2:0.085828  MSE:0.003642  KL:2.053928  MAE:0.011799  RMSE:0.060349  CVRMSE:15.087372  R2:0.045529  MSE:0.003803  KL:2.555929  MAE:0.012466  RMSE:0.061665  CVRMSE:15.416329  R2:0.081976  MSE:0.003657  KL:2.604995  MAE:0.012314  RMSE:0.060476  CVRMSE:15.119123  train_loss:
0.0036574056051904336
R2:0.063152  MSE:0.002802  KL:1.359929  MAE:0.010644  RMSE:0.052935  CVRMSE:17.644990  test_loss:
1.4010555823915637e-05
R2:0.076547  MSE:0.003679  KL:1.738804  MAE:0.012550  RMSE:0.060655  CVRMSE:15.163766  R2:0.091242  MSE:0.003620  KL:2.227534  MAE:0.012932  RMSE:0.060171  CVRMSE:15.042633  R2:0.075062  MSE:0.003685  KL:1.859187  MAE:0.013302  RMSE:0.060704  CVRMSE:15.175952  R2:0.048309  MSE:0.003792  KL:2.102163  MAE:0.013757  RMSE:0.061575  CVRMSE:15.393868  R2:0.056083  MSE:0.003761  KL:1.879141  MAE:0.012742  RMSE:0.0

R2:0.061536  MSE:0.003739  KL:2.009654  MAE:0.011585  RMSE:0.061146  CVRMSE:15.286513  R2:0.065561  MSE:0.003723  KL:2.253682  MAE:0.011012  RMSE:0.061015  CVRMSE:15.253694  R2:0.074196  MSE:0.003688  KL:1.271068  MAE:0.010982  RMSE:0.060732  CVRMSE:15.183054  R2:0.088513  MSE:0.003631  KL:1.318648  MAE:0.011258  RMSE:0.060261  CVRMSE:15.065198  R2:0.076339  MSE:0.003680  KL:1.663432  MAE:0.011487  RMSE:0.060662  CVRMSE:15.165474  train_loss:
0.0036798652072320692
R2:0.071642  MSE:0.002777  KL:2.256644  MAE:0.008974  RMSE:0.052695  CVRMSE:17.564864  test_loss:
1.3883600535336881e-05
R2:0.071019  MSE:0.003701  KL:2.397695  MAE:0.011505  RMSE:0.060836  CVRMSE:15.209084  R2:0.082415  MSE:0.003656  KL:2.140418  MAE:0.011850  RMSE:0.060462  CVRMSE:15.115508  R2:0.075781  MSE:0.003682  KL:1.902803  MAE:0.011849  RMSE:0.060680  CVRMSE:15.170053  R2:0.074117  MSE:0.003689  KL:1.740508  MAE:0.011571  RMSE:0.060735  CVRMSE:15.183704  R2:0.082990  MSE:0.003653  KL:1.750887  MAE:0.011736  RMSE:0.0

R2:0.058339  MSE:0.003752  KL:1.417276  MAE:0.011152  RMSE:0.061250  CVRMSE:15.312531  R2:0.051720  MSE:0.003778  KL:1.253885  MAE:0.010617  RMSE:0.061465  CVRMSE:15.366249  R2:0.075091  MSE:0.003685  KL:1.510083  MAE:0.010690  RMSE:0.060703  CVRMSE:15.175716  R2:0.052969  MSE:0.003773  KL:1.330129  MAE:0.011195  RMSE:0.061425  CVRMSE:15.356132  R2:0.079825  MSE:0.003666  KL:1.072021  MAE:0.010448  RMSE:0.060547  CVRMSE:15.136832  train_loss:
0.0036659790755948054
R2:0.071316  MSE:0.002778  KL:0.376055  MAE:0.010049  RMSE:0.052704  CVRMSE:17.567946  test_loss:
1.3888472414691933e-05
R2:0.065618  MSE:0.003723  KL:1.472733  MAE:0.011170  RMSE:0.061013  CVRMSE:15.253237  R2:0.059306  MSE:0.003748  KL:1.575106  MAE:0.011285  RMSE:0.061219  CVRMSE:15.304669  R2:0.066564  MSE:0.003719  KL:0.757120  MAE:0.010005  RMSE:0.060982  CVRMSE:15.245511  R2:0.077082  MSE:0.003677  KL:1.437039  MAE:0.011099  RMSE:0.060637  CVRMSE:15.159369  R2:0.056082  MSE:0.003761  KL:1.452230  MAE:0.011353  RMSE:0.0

...

R2:0.070704  MSE:0.003702  KL:0.912556  MAE:0.009632  RMSE:0.060847  CVRMSE:15.211666  R2:0.079176  MSE:0.003669  KL:1.203029  MAE:0.009391  RMSE:0.060569  CVRMSE:15.142162  R2:0.079002  MSE:0.003669  KL:0.484180  MAE:0.009451  RMSE:0.060574  CVRMSE:15.143596  R2:0.080152  MSE:0.003665  KL:0.751964  MAE:0.009693  RMSE:0.060537  CVRMSE:15.134142  R2:0.081421  MSE:0.003660  KL:1.046709  MAE:0.009568  RMSE:0.060495  CVRMSE:15.123699  train_loss:
0.0036596204896341077
R2:0.075098  MSE:0.002766  KL:2.684044  MAE:0.008560  RMSE:0.052596  CVRMSE:17.532136  test_loss:
1.3831910255248658e-05
R2:0.086227  MSE:0.003640  KL:0.771580  MAE:0.010210  RMSE:0.060336  CVRMSE:15.084084  R2:0.073691  MSE:0.003690  KL:0.740266  MAE:0.009877  RMSE:0.060749  CVRMSE:15.187195  R2:0.088736  MSE:0.003630  KL:1.095699  MAE:0.010118  RMSE:0.060253  CVRMSE:15.063361  R2:0.070949  MSE:0.003701  KL:2.012112  MAE:0.009702  RMSE:0.060839  CVRMSE:15.209661  R2:0.073229  MSE:0.003692  KL:0.824736  MAE:0.009729  RMSE:0.0

R2:0.088873  MSE:0.003630  KL:1.107552  MAE:0.009534  RMSE:0.060249  CVRMSE:15.062222  R2:0.077192  MSE:0.003676  KL:1.638042  MAE:0.009855  RMSE:0.060634  CVRMSE:15.158467  R2:0.075959  MSE:0.003681  KL:0.557669  MAE:0.009772  RMSE:0.060674  CVRMSE:15.168595  R2:0.072636  MSE:0.003695  KL:1.406523  MAE:0.009726  RMSE:0.060783  CVRMSE:15.195847  R2:0.075185  MSE:0.003684  KL:0.968908  MAE:0.009707  RMSE:0.060700  CVRMSE:15.174945  train_loss:
0.003684463200625032
R2:0.076452  MSE:0.002762  KL:2.123804  MAE:0.008122  RMSE:0.052558  CVRMSE:17.519294  test_loss:
1.3811654614983126e-05
R2:0.059216  MSE:0.003748  KL:0.941630  MAE:0.009813  RMSE:0.061222  CVRMSE:15.305398  R2:0.065936  MSE:0.003721  KL:0.670094  MAE:0.009288  RMSE:0.061003  CVRMSE:15.250639  R2:0.058913  MSE:0.003749  KL:2.922605  MAE:0.009297  RMSE:0.061231  CVRMSE:15.307863  R2:0.076699  MSE:0.003678  KL:0.911395  MAE:0.009190  RMSE:0.060650  CVRMSE:15.162522  R2:0.076213  MSE:0.003680  KL:1.644114  MAE:0.009132  RMSE:0.06

R2:0.083182  MSE:0.003653  KL:1.807029  MAE:0.009771  RMSE:0.060437  CVRMSE:15.109191  R2:0.077840  MSE:0.003674  KL:1.888180  MAE:0.010038  RMSE:0.060613  CVRMSE:15.153143  R2:0.084574  MSE:0.003647  KL:0.792415  MAE:0.009857  RMSE:0.060391  CVRMSE:15.097715  R2:0.078948  MSE:0.003669  KL:1.509077  MAE:0.009886  RMSE:0.060576  CVRMSE:15.144041  R2:0.082988  MSE:0.003653  KL:0.662187  MAE:0.009974  RMSE:0.060443  CVRMSE:15.110789  train_loss:
0.003653375504654832
R2:0.076951  MSE:0.002761  KL:2.605620  MAE:0.007993  RMSE:0.052544  CVRMSE:17.514560  test_loss:
1.3804190413793548e-05
R2:0.068371  MSE:0.003712  KL:0.581270  MAE:0.009700  RMSE:0.060923  CVRMSE:15.230750  R2:0.066682  MSE:0.003718  KL:0.589206  MAE:0.009743  RMSE:0.060978  CVRMSE:15.244543  R2:0.094617  MSE:0.003607  KL:1.200696  MAE:0.009666  RMSE:0.060059  CVRMSE:15.014675  R2:0.074782  MSE:0.003686  KL:2.654540  MAE:0.009629  RMSE:0.060713  CVRMSE:15.178251  R2:0.094865  MSE:0.003606  KL:1.873257  MAE:0.009554  RMSE:0.06

R2:0.092775  MSE:0.003614  KL:0.547981  MAE:0.009162  RMSE:0.060120  CVRMSE:15.029935  R2:0.071439  MSE:0.003699  KL:1.351894  MAE:0.009594  RMSE:0.060823  CVRMSE:15.205649  R2:0.079680  MSE:0.003667  KL:1.490936  MAE:0.009556  RMSE:0.060552  CVRMSE:15.138020  R2:0.084971  MSE:0.003645  KL:0.716499  MAE:0.009765  RMSE:0.060378  CVRMSE:15.094447  R2:0.066989  MSE:0.003717  KL:0.656775  MAE:0.009742  RMSE:0.060968  CVRMSE:15.242038  train_loss:
0.003717115122708492
R2:0.077014  MSE:0.002761  KL:2.332803  MAE:0.007935  RMSE:0.052542  CVRMSE:17.513962  test_loss:
1.380324964702595e-05
R2:0.078677  MSE:0.003671  KL:1.944178  MAE:0.009549  RMSE:0.060585  CVRMSE:15.146270  R2:0.075598  MSE:0.003683  KL:0.794287  MAE:0.009492  RMSE:0.060686  CVRMSE:15.171554  R2:0.093655  MSE:0.003611  KL:1.970718  MAE:0.009592  RMSE:0.060091  CVRMSE:15.022650  R2:0.069386  MSE:0.003708  KL:1.234006  MAE:0.009708  RMSE:0.060890  CVRMSE:15.222444  R2:0.072010  MSE:0.003697  KL:0.586908  MAE:0.009769  RMSE:0.060

R2:0.077116  MSE:0.003677  KL:1.313480  MAE:0.009704  RMSE:0.060636  CVRMSE:15.159091  R2:0.076000  MSE:0.003681  KL:2.670909  MAE:0.009638  RMSE:0.060673  CVRMSE:15.168255  R2:0.069227  MSE:0.003708  KL:0.859866  MAE:0.009715  RMSE:0.060895  CVRMSE:15.223744  R2:0.070212  MSE:0.003704  KL:2.651074  MAE:0.009684  RMSE:0.060863  CVRMSE:15.215690  R2:0.073652  MSE:0.003691  KL:1.016216  MAE:0.009405  RMSE:0.060750  CVRMSE:15.187516  train_loss:
0.0036905699846101923
R2:0.076964  MSE:0.002761  KL:2.247942  MAE:0.007642  RMSE:0.052543  CVRMSE:17.514442  test_loss:
1.3804005851852708e-05
R2:0.082217  MSE:0.003656  KL:0.439661  MAE:0.009252  RMSE:0.060469  CVRMSE:15.117145  R2:0.089794  MSE:0.003626  KL:1.186621  MAE:0.009444  RMSE:0.060218  CVRMSE:15.054614  R2:0.080344  MSE:0.003664  KL:1.121370  MAE:0.009506  RMSE:0.060530  CVRMSE:15.132560  R2:0.085954  MSE:0.003642  KL:0.799144  MAE:0.009576  RMSE:0.060345  CVRMSE:15.086335  R2:0.076487  MSE:0.003679  KL:0.923383  MAE:0.009791  RMSE:0.0

In [ ]:
# estimate area under the uplift curve (AUUC)
uplift=df_test.copy()
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T', treatment_effect_col='tau')
gat_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gat_auuc

,auuc
y_hat,0.830929
Random,0.489027


In [ ]:
print('Feature mode on GAT:', feats_mode)
print('MSE:',MSE(uplift['y'],uplift['y_hat']))

MSE: 0.0027703335521529373


In [ ]:
if feats_mode == 'imp':
    result.loc['GAT (Struct+Important Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'causal+imp':
    result.loc['GAT (Struct+Causal+Important Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'causal*imp':
    result.loc['GAT (Struct+(Causal*Important) Weighting','AUUC']=auuc["y_hat"]
result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807
GCN (Struct+Causal+Important Weighting),0.843556


# GAT
Built-in Pytorch Geometric with default attention matrix without Bayesian Network Connectivity

In [ ]:
edge_index=pd.DataFrame(columns=['start','end'])
edge_index['start']=[i+1 for i in range(14)]
edge_index['end']=[i+1 for i in range(14)]
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

edge_index

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]])

In [ ]:
# Input the feature mode and get corresponding dimensions

feats_mode='causal+imp'


if feats_mode=='causal+imp':
    in_dim=39  # 1+10+14+14

elif feats_mode =='noweighting':
    in_dim=11  # 1+10

elif feats_mode in ['causal*imp','causal','imp','equal']:
    in_dim=25  # 1+10+14
    
else:
    print('Error: Input Dimension Error')
    
# subset data for demo, delete this while training ob full dataset
df_train = df_train[:1000]
df_test = df_test[:1000]

col=df_train.columns


data_list=[]
weighted_feats=[]

# Set no. of folds on dataset
folds= 10  # 10 folds
batch=int(len(df_train)/folds) # batch size 

for f in range(folds):
    train=df_train[(batch*f):(batch*(f+1))]
    x_train=torch.from_numpy(np.array(train[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
    y_train=torch.from_numpy(np.array(train['y'])).reshape(train.shape[0],1).to(torch.float32)

    for i in range(x_train.shape[0]):
        Edge_index = edge_index.type(torch.long)
        X =x_train[i]
        
        if feats_mode =='causal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted) # feature (10d), causally weighted features        
                                                    ))) 
        elif feats_mode =='imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            weighted_feats.append(imp_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(imp_weighted) # feature (10d),importance weighted features        
                                                    ))) 
        elif feats_mode =='causal*imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.multiply(np.multiply(np.array(X),np.array(imp_list)),np.array(ate_list))
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='causal+imp':
            t=torch.zeros(14,39) # 39 dimensions = 1+10+14+14 = node number, node embedding, causal features, important features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(imp_weighted+causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted),# feature (10d), causality weighted features
                                                    list(imp_weighted)# feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='equal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.array(X)
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), equally weighted features        
                                                    ))) 
        else:
            t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


            for j in range(14):
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    ))) 

        Y = y_train[i].reshape(-1,1).to(torch.float32)
        data = Data(x=t, edge_index=Edge_index, y=Y)
        data_list.append(data)
train_loader =DataLoader(data_list, batch_size=200, shuffle=False,num_workers=0)


y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)

data_list=[]
weighted_feats=[]

for f in range(folds):

    test=df_test[(batch*f):(batch*(f+1))]
    x_test=torch.from_numpy(np.array(test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
    y_test=torch.from_numpy(np.array(test['y'])).reshape(test.shape[0],1).to(torch.float32)

    for i in range(x_test.shape[0]):
        Edge_index = edge_index.type(torch.long)
        X =x_test[i]

        if feats_mode =='causal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted) # feature (10d), causally weighted features        
                                                    ))) 
        elif feats_mode =='imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            weighted_feats.append(imp_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(imp_weighted) # feature (10d),importance weighted features        
                                                    ))) 
        elif feats_mode =='causal*imp':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.multiply(np.multiply(np.array(X),np.array(imp_list)),np.array(ate_list))
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='causal+imp':
            t=torch.zeros(14,39) # 39 dimensions = 1+10+14+14 = node number, node embedding, causal features, important features

            imp_weighted= np.multiply(np.array(X),np.array(imp_list))
            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(imp_weighted+causal_weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(causal_weighted),# feature (10d), causality weighted features
                                                    list(imp_weighted)# feature (10d), importance weighted features        
                                                    ))) 
        elif feats_mode =='equal':
            t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

            weighted= np.array(X)
            weighted_feats.append(weighted)

            for j in range(14): # for j-th feature
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    list(weighted) # feature (10d), equally weighted features        
                                                    ))) 
        else:
            t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


            for j in range(14):
                t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                    list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                    ))) 

        Y = y_test[i].reshape(-1,1).to(torch.float32)
        data = Data(x=t, edge_index=Edge_index, y=Y)
        data_list.append(data)
test_loader =DataLoader(data_list, batch_size=200, shuffle=False,num_workers=0)

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=in_dim,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(train_loader):
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(df_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in iter(test_loader):
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(df_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl



num_epochs = 2560
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in iter(test_loader):
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-85.316482  MSE:0.343885  KL:2.721755  MAE:0.426810  RMSE:0.586417  CVRMSE:146.604242  train_loss:
0.3438848674297333
R2:-0.116688  MSE:0.003340  KL:2.672313  MAE:0.016765  RMSE:0.057793  CVRMSE:19.264286  test_loss:
1.67000723304227e-05
R2:-25.708243  MSE:0.106406  KL:2.699752  MAE:0.243195  RMSE:0.326199  CVRMSE:81.549694  R2:-15.054149  MSE:0.063960  KL:2.782090  MAE:0.185219  RMSE:0.252903  CVRMSE:63.225652  R2:-8.684555  MSE:0.038583  KL:2.845542  MAE:0.142577  RMSE:0.196426  CVRMSE:49.106560  R2:-4.573907  MSE:0.022206  KL:2.933564  MAE:0.102578  RMSE:0.149018  CVRMSE:37.254567  R2:-3.834650  MSE:0.019261  KL:3.019548  MAE:0.095120  RMSE:0.138785  CVRMSE:34.696224  train_loss:
0.019261247664690017
R2:-0.096358  MSE:0.003279  KL:4.352970  MAE:0.018290  RMSE:0.057264  CVRMSE:19.088119  test_loss:
1.6396035527577625e-05
R2:-2.103998  MSE:0.012366  KL:3.013953  MAE:0.077458  RMSE:0.111204  CVRMSE:27.800999  R2:-1.334576  MSE:0.009301  KL:3.128081  MAE:0.063581  RMSE:0.096441  CVRM

R2:-0.110161  MSE:0.004423  KL:2.983040  MAE:0.022371  RMSE:0.066505  CVRMSE:16.626183  R2:-0.166780  MSE:0.004648  KL:3.085367  MAE:0.022277  RMSE:0.068180  CVRMSE:17.044887  R2:-0.052469  MSE:0.004193  KL:2.953662  MAE:0.022898  RMSE:0.064754  CVRMSE:16.188418  R2:-0.035050  MSE:0.004124  KL:3.093516  MAE:0.024888  RMSE:0.064216  CVRMSE:16.053893  R2:-0.063143  MSE:0.004236  KL:3.029905  MAE:0.022221  RMSE:0.065081  CVRMSE:16.270299  train_loss:
0.0042355618672445415
R2:0.062870  MSE:0.002803  KL:0.907885  MAE:0.007426  RMSE:0.052943  CVRMSE:17.647646  test_loss:
1.401477454783162e-05
R2:-0.083659  MSE:0.004317  KL:2.977087  MAE:0.021282  RMSE:0.065706  CVRMSE:16.426533  R2:-0.071701  MSE:0.004270  KL:2.834494  MAE:0.020797  RMSE:0.065343  CVRMSE:16.335653  R2:-0.014515  MSE:0.004042  KL:3.276501  MAE:0.022445  RMSE:0.063575  CVRMSE:15.893843  R2:-0.130863  MSE:0.004505  KL:3.072630  MAE:0.025306  RMSE:0.067122  CVRMSE:16.780495  R2:0.053323  MSE:0.003772  KL:2.838896  MAE:0.021799  

R2:-0.039364  MSE:0.004141  KL:2.728677  MAE:0.018998  RMSE:0.064349  CVRMSE:16.087313  R2:-0.079586  MSE:0.004301  KL:3.077791  MAE:0.020627  RMSE:0.065583  CVRMSE:16.395634  R2:0.060919  MSE:0.003741  KL:2.839260  MAE:0.018947  RMSE:0.061166  CVRMSE:15.291539  R2:-0.052275  MSE:0.004192  KL:2.959470  MAE:0.021894  RMSE:0.064748  CVRMSE:16.186926  R2:-0.015086  MSE:0.004044  KL:2.816052  MAE:0.019051  RMSE:0.063593  CVRMSE:15.898318  train_loss:
0.004044104128843174
R2:0.067353  MSE:0.002790  KL:1.067622  MAE:0.009588  RMSE:0.052816  CVRMSE:17.605388  test_loss:
1.394773632637225e-05
R2:-0.063957  MSE:0.004239  KL:2.943142  MAE:0.022124  RMSE:0.065106  CVRMSE:16.276529  R2:-0.067508  MSE:0.004253  KL:2.933718  MAE:0.021986  RMSE:0.065215  CVRMSE:16.303668  R2:-0.107601  MSE:0.004413  KL:3.096445  MAE:0.020844  RMSE:0.066428  CVRMSE:16.607004  R2:-0.075733  MSE:0.004286  KL:2.967427  MAE:0.024347  RMSE:0.065465  CVRMSE:16.366350  R2:-0.130242  MSE:0.004503  KL:2.953884  MAE:0.021570  R

R2:0.007444  MSE:0.003954  KL:2.687121  MAE:0.017704  RMSE:0.062884  CVRMSE:15.720892  R2:-0.017262  MSE:0.004053  KL:2.659385  MAE:0.018943  RMSE:0.063661  CVRMSE:15.915346  R2:0.021258  MSE:0.003899  KL:2.635626  MAE:0.017917  RMSE:0.062444  CVRMSE:15.611108  R2:-0.008295  MSE:0.004017  KL:2.963068  MAE:0.017388  RMSE:0.063380  CVRMSE:15.845047  R2:0.002502  MSE:0.003974  KL:2.858534  MAE:0.017322  RMSE:0.063040  CVRMSE:15.759980  train_loss:
0.003974031552206725
R2:0.064887  MSE:0.002797  KL:3.936264  MAE:0.010085  RMSE:0.052886  CVRMSE:17.628645  test_loss:
1.3984609613544308e-05
R2:-0.002705  MSE:0.003995  KL:2.654734  MAE:0.016967  RMSE:0.063204  CVRMSE:15.801060  R2:0.006568  MSE:0.003958  KL:2.728623  MAE:0.016722  RMSE:0.062911  CVRMSE:15.727831  R2:0.050084  MSE:0.003784  KL:2.728821  MAE:0.017949  RMSE:0.061518  CVRMSE:15.379502  R2:-0.024208  MSE:0.004080  KL:2.646087  MAE:0.016470  RMSE:0.063878  CVRMSE:15.969591  R2:0.048019  MSE:0.003793  KL:2.842224  MAE:0.015776  RMSE:

R2:0.042903  MSE:0.003813  KL:2.346539  MAE:0.014015  RMSE:0.061750  CVRMSE:15.437522  R2:0.063279  MSE:0.003732  KL:2.626891  MAE:0.014058  RMSE:0.061089  CVRMSE:15.272315  R2:0.092267  MSE:0.003616  KL:2.973653  MAE:0.016132  RMSE:0.060137  CVRMSE:15.034144  R2:0.039112  MSE:0.003828  KL:2.726526  MAE:0.016087  RMSE:0.061872  CVRMSE:15.468068  R2:0.078625  MSE:0.003671  KL:2.788879  MAE:0.015023  RMSE:0.060587  CVRMSE:15.146691  train_loss:
0.0036707561172079294
R2:0.071392  MSE:0.002777  KL:0.115838  MAE:0.008770  RMSE:0.052702  CVRMSE:17.567224  test_loss:
1.3887331209843978e-05
R2:0.036200  MSE:0.003840  KL:2.451581  MAE:0.014923  RMSE:0.061966  CVRMSE:15.491486  R2:-0.016997  MSE:0.004052  KL:2.397114  MAE:0.015132  RMSE:0.063653  CVRMSE:15.913273  R2:0.028873  MSE:0.003869  KL:2.390065  MAE:0.015050  RMSE:0.062201  CVRMSE:15.550258  R2:0.063515  MSE:0.003731  KL:2.785472  MAE:0.014534  RMSE:0.061082  CVRMSE:15.270387  R2:0.037103  MSE:0.003836  KL:2.619903  MAE:0.015928  RMSE:0.

...

R2:0.121402  MSE:0.003500  KL:1.379493  MAE:0.008308  RMSE:0.059164  CVRMSE:14.790909  R2:0.107329  MSE:0.003556  KL:0.320819  MAE:0.009181  RMSE:0.059636  CVRMSE:14.908890  R2:-0.011449  MSE:0.004030  KL:0.644449  MAE:0.010095  RMSE:0.063479  CVRMSE:15.869808  R2:0.067867  MSE:0.003714  KL:0.313115  MAE:0.009312  RMSE:0.060939  CVRMSE:15.234865  R2:0.090566  MSE:0.003623  KL:4.443797  MAE:0.010470  RMSE:0.060193  CVRMSE:15.048225  train_loss:
0.003623185437754728
R2:0.065100  MSE:0.002796  KL:0.085546  MAE:0.009078  RMSE:0.052880  CVRMSE:17.626638  test_loss:
1.3981425625388511e-05
R2:0.077449  MSE:0.003675  KL:0.590933  MAE:0.009072  RMSE:0.060625  CVRMSE:15.156354  R2:0.039980  MSE:0.003825  KL:2.575139  MAE:0.008535  RMSE:0.061844  CVRMSE:15.461077  R2:0.114567  MSE:0.003528  KL:0.193281  MAE:0.008291  RMSE:0.059393  CVRMSE:14.848330  R2:0.059333  MSE:0.003748  KL:3.624522  MAE:0.009537  RMSE:0.061218  CVRMSE:15.304446  R2:0.070041  MSE:0.003705  KL:1.557528  MAE:0.008991  RMSE:0.0

R2:0.071042  MSE:0.003701  KL:0.643650  MAE:0.009270  RMSE:0.060836  CVRMSE:15.208896  R2:0.043836  MSE:0.003809  KL:0.287170  MAE:0.008692  RMSE:0.061720  CVRMSE:15.430000  R2:0.045612  MSE:0.003802  KL:0.951391  MAE:0.009057  RMSE:0.061663  CVRMSE:15.415660  R2:0.050990  MSE:0.003781  KL:0.413661  MAE:0.008841  RMSE:0.061489  CVRMSE:15.372167  R2:0.104680  MSE:0.003567  KL:0.461448  MAE:0.009608  RMSE:0.059724  CVRMSE:14.931000  train_loss:
0.0035669565026182682
R2:0.061072  MSE:0.002808  KL:0.107678  MAE:0.010058  RMSE:0.052994  CVRMSE:17.664574  test_loss:
1.4041672882740385e-05
R2:0.088445  MSE:0.003632  KL:0.295389  MAE:0.009955  RMSE:0.060263  CVRMSE:15.065762  R2:0.070647  MSE:0.003703  KL:0.324241  MAE:0.008784  RMSE:0.060849  CVRMSE:15.212130  R2:0.035858  MSE:0.003841  KL:0.422809  MAE:0.009323  RMSE:0.061977  CVRMSE:15.494233  R2:0.025035  MSE:0.003884  KL:2.177576  MAE:0.008466  RMSE:0.062324  CVRMSE:15.580958  R2:0.109694  MSE:0.003547  KL:0.442515  MAE:0.008247  RMSE:0.0

R2:0.066790  MSE:0.003718  KL:5.197562  MAE:0.010228  RMSE:0.060975  CVRMSE:15.243660  R2:0.051152  MSE:0.003780  KL:0.821095  MAE:0.008477  RMSE:0.061483  CVRMSE:15.370855  R2:0.054480  MSE:0.003767  KL:3.440649  MAE:0.008449  RMSE:0.061376  CVRMSE:15.343875  R2:0.064914  MSE:0.003725  KL:0.486185  MAE:0.008968  RMSE:0.061036  CVRMSE:15.258975  R2:0.050743  MSE:0.003782  KL:0.953128  MAE:0.008709  RMSE:0.061497  CVRMSE:15.374163  train_loss:
0.003781838115537539
R2:0.066428  MSE:0.002792  KL:0.097889  MAE:0.006343  RMSE:0.052842  CVRMSE:17.614113  test_loss:
1.3961564174678643e-05
R2:0.045806  MSE:0.003802  KL:0.371011  MAE:0.008991  RMSE:0.061656  CVRMSE:15.414095  R2:0.071018  MSE:0.003701  KL:3.530188  MAE:0.009095  RMSE:0.060836  CVRMSE:15.209090  R2:0.057942  MSE:0.003753  KL:0.323252  MAE:0.009389  RMSE:0.061263  CVRMSE:15.315761  R2:0.063674  MSE:0.003730  KL:1.173493  MAE:0.008879  RMSE:0.061076  CVRMSE:15.269088  R2:0.060966  MSE:0.003741  KL:0.357154  MAE:0.009796  RMSE:0.06

R2:0.092128  MSE:0.003617  KL:0.373534  MAE:0.009382  RMSE:0.060141  CVRMSE:15.035295  R2:0.076413  MSE:0.003680  KL:2.905778  MAE:0.009445  RMSE:0.060659  CVRMSE:15.164863  R2:0.055442  MSE:0.003763  KL:0.677907  MAE:0.009352  RMSE:0.061344  CVRMSE:15.336065  R2:0.058172  MSE:0.003752  KL:0.963593  MAE:0.008686  RMSE:0.061256  CVRMSE:15.313889  R2:0.048891  MSE:0.003789  KL:0.575346  MAE:0.009367  RMSE:0.061557  CVRMSE:15.389153  train_loss:
0.003789216351287905
R2:0.067548  MSE:0.002789  KL:0.090699  MAE:0.007885  RMSE:0.052811  CVRMSE:17.603552  test_loss:
1.3944826627266593e-05
R2:0.045290  MSE:0.003804  KL:0.783272  MAE:0.008843  RMSE:0.061673  CVRMSE:15.418258  R2:0.083115  MSE:0.003653  KL:1.415706  MAE:0.007949  RMSE:0.060439  CVRMSE:15.109743  R2:0.050998  MSE:0.003781  KL:1.692590  MAE:0.008347  RMSE:0.061488  CVRMSE:15.372098  R2:0.029017  MSE:0.003868  KL:2.731953  MAE:0.008626  RMSE:0.062196  CVRMSE:15.549107  R2:0.051978  MSE:0.003777  KL:1.493638  MAE:0.008679  RMSE:0.06

R2:0.064275  MSE:0.003728  KL:3.723473  MAE:0.009685  RMSE:0.061057  CVRMSE:15.264194  R2:0.073252  MSE:0.003692  KL:2.702214  MAE:0.009207  RMSE:0.060763  CVRMSE:15.190792  R2:0.065320  MSE:0.003724  KL:0.752980  MAE:0.009071  RMSE:0.061023  CVRMSE:15.255666  R2:0.101009  MSE:0.003582  KL:1.283122  MAE:0.008819  RMSE:0.059846  CVRMSE:14.961580  R2:0.041530  MSE:0.003819  KL:3.572134  MAE:0.009917  RMSE:0.061794  CVRMSE:15.448594  train_loss:
0.0038185446901479736
R2:0.070825  MSE:0.002779  KL:0.087214  MAE:0.006225  RMSE:0.052718  CVRMSE:17.572590  test_loss:
1.3895817115553654e-05
R2:0.043856  MSE:0.003809  KL:0.628827  MAE:0.009361  RMSE:0.061719  CVRMSE:15.429838  R2:0.114098  MSE:0.003529  KL:2.659083  MAE:0.008843  RMSE:0.059409  CVRMSE:14.852260  R2:0.117797  MSE:0.003515  KL:1.122354  MAE:0.009723  RMSE:0.059285  CVRMSE:14.821221  R2:0.040036  MSE:0.003824  KL:2.206689  MAE:0.008856  RMSE:0.061843  CVRMSE:15.460628  R2:0.047042  MSE:0.003797  KL:2.004125  MAE:0.008642  RMSE:0.0

In [ ]:
# estimate area under the uplift curve (AUUC)
uplift=df_test.copy()
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T', treatment_effect_col='tau')
gat_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gat_auuc

,auuc
y_hat,0.590529
Random,0.489027


In [ ]:
print('Feature mode on GAT without BN connectivity:', feats_mode)
print('MSE:',MSE(uplift['y'],uplift['y_hat']))

Feature mode on GAT without BN connectivity: causal+imp
MSE: 0.002852495611763611


In [ ]:
if feats_mode == 'causal':
    result.loc['GAT (Causal Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'imp':
    result.loc['GAT (Important Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'equal':
    result.loc['GAT (Feature)','AUUC']=auuc["y_hat"]
if feats_mode == 'noweighting':
    result.loc['GAT','AUUC']=auuc["y_hat"]

if feats_mode == 'causal+imp':
    result.loc['GAT (Causal+Important Weighting)','AUUC']=auuc["y_hat"]
if feats_mode == 'causal*imp':
    result.loc['GAT (Causal*Important) Weighting','AUUC']=auuc["y_hat"]
result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807
GCN (Struct+Causal+Important Weighting),0.777582
